<a href="https://colab.research.google.com/github/Shezan57/LLM/blob/main/song_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from urllib import request

In [2]:
# Get the playlist datasets file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

In [3]:
# Parse the playlist dataset file. skip the first two lines as they only contain metadata
lines = data.read().decode('utf-8').split('\n')[2:]

In [7]:
# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

In [9]:
# Load song metadata
# Original: songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes/yes_complete/song_hash.txt')
# The URL might be incorrect or the file might require specific permissions to access.
# Consider checking if the URL is accessible publicly.
# If the file is restricted, ensure you have the necessary credentials or permissions to download it.
# For example, if it is hosted on Google Cloud Storage, you might need to set up authentication using gcloud.

# Try using the following URL, which has been observed to work:
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
sons_file = songs_file.read().decode('utf-8').split('\n')
# Correct the typo: 'rstirp' should be 'rstrip'
songs = [s.rstrip().split('\t') for s in sons_file]
songs_df = pd.DataFrame(data=songs, columns=['id','title','artist'])
songs_df = songs_df.set_index('id')

In [10]:
print('playlist #1:\n ', playlists[0], '\n')
print('playlist #2:\n ', playlists[1])

playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [11]:
from gensim.models import Word2Vec

In [12]:
# Train our Word2Vec model
model = Word2Vec(playlists,vector_size=32,window=20,negative=50,min_count=1,workers=4)

In [13]:
song_id = 2172
# Ask the model for songs similar to song 2172
model.wv.most_similar(positive=str(song_id))

[('2014', 0.9966034293174744),
 ('2849', 0.9961791634559631),
 ('5586', 0.9954848885536194),
 ('3167', 0.9945949912071228),
 ('1922', 0.9940653443336487),
 ('6641', 0.9936787486076355),
 ('6658', 0.9933788776397705),
 ('2976', 0.9930082559585571),
 ('6624', 0.9925915598869324),
 ('11477', 0.9925079345703125)]

In [14]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [28]:
print(songs_df.iloc[2014])

title     Youth Gone Wild
artist           Skid Row
Name: 2014 , dtype: object


In [16]:
import numpy as np

In [35]:
def print_recommendations(song_id):
  similar_songs = np.array(
      model.wv.most_similar(positive=str(song_id), topn=5)
  )[:,0]
  similar_songs = similar_songs.astype(int)
  # Fileter similar songs to keep only those present in the index
  similar_songs = [song for song in similar_songs if song in songs_df.index]
  # Check if any similar songs remain after filtering
  if similar_songs:
    return songs_df.loc[similar_songs]
    # If no similar songs remain, return a message
  else:
    print("No similar songs found.")
    return pd.DataFrame()

In [36]:
print_recommendations(2014)

No similar songs found.


""
